In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
from glob import glob

import matplotlib.pyplot as plt

import rioxarray as rio
from shapely.geometry import mapping

%matplotlib inline
# %matplotlib widget

In [ ]:
# import era5 data 
# merge all together 
# clip by HMA regions (pick one) 
# find median pixel (location)
# go to GEE find the date of snowline in that location (average over pixel area maybe?)
# 

### read hma regions

In [ ]:
hma = gpd.read_file('HMA_regions/HMA_regions.shp')
# hma.plot()

# as dataframe
hma_df = pd.DataFrame(hma) 
# hma_df.columns
hma_df.Name[1]

## era5 for 2012 as an example! 

In [ ]:
'''
# read:
t2m = xr.open_dataset('era5_2m-temperature_hourly_2012.nc', decode_coords="all")
ssrd = xr.open_dataset('era5_surface-solar-radiation-downwards_hourly_2012.nc', decode_coords="all")
tcc = xr.open_dataset('era5_total-cloud-cover_hourly_2012.nc', decode_coords="all")
tp = xr.open_dataset('era5_total-precipitation_hourly_2012.nc', decode_coords="all")

# merge by coordinates 
# tutorial: https://docs.xarray.dev/en/stable/user-guide/combining.html
climate = xr.merge([t2m, tp, ssrd, tcc]) 
'''

### OR: era5 open all 

In [ ]:
climate = xr.open_mfdataset('*.nc', decode_coords="all")
# cnange units (and overwrite the metadata with the units after)

#convert temperature K to C
climate['t2m']=climate.t2m-273.15
# precipotation m to mm 
climate['tp']=climate.tp * 1000
# radiation j/m2 to w/m2
# SSR [W/m2] = SSR [J/m^2] / (3600 seconds)
climate['ssrd'] = climate.ssrd / 3600

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
climate.t2m[1,:,:].plot()#palette='viridis')
hma.boundary.plot(ax=ax, alpha=.6, color='black')
fig.tight_layout()

In [ ]:
''' change name of the mountain range here'''

roi = hma[hma['Name'] == hma_df.Name[21]]
name = roi.Name
print(roi.Name)
# karakoram.crs

In [ ]:
# add crs to the dataset (netcdf file) - maybe this is not even needed 
climate = climate.rio.write_crs('EPSG:4326')
 
# clip using xarrayrio library:
roi_climate = climate.rio.clip(roi.geometry, roi.crs, drop=True, invert=False)

In [ ]:
'''
# see how it looks like 
roi_climate.t2m[795,:,:].plot()
fig.tight_layout()
'''

In [ ]:
'''
# calculate how many era5 pix fall into the mountain range polygon

x = roi_climate.t2m[1,:,:].values.flatten()
print('total numnber of pixels within bbox:', x.shape, '\n')

# without nans
x2 = x[~np.isnan(x)]
print('number of not-nan pixels:', x2.shape, '\n')


# check what to do for the median (in space) value: 
if (len(x2) % 2) == 0:
    print ('EVEN number of not-nans -> ceiling/floor median')
else:print ('ODD number of not-nans -> regular median is ok')
'''

### time dimention (-> value IN TIME)

In [ ]:
## "flatten" the data over time: reduce to smth (median temperatures/sum of precipitation over the year/etc) 

# statistics + keep the coordinates -> value IN TIME 
roi_climate_median = roi_climate.median(dim='time', keep_attrs = True, skipna=True)
# roi_climate_sum    = roi_climate.sum(dim='time', keep_attrs = True)


### space dimention (with pandas)

In [ ]:
# with pandas: 
# (1) compute median in time (-> on the previous step) 
# (2) convert to dataframe (with coords being index)
# (3) kick out nans and duplicates
# (4) compute median (or some other thing) across the table 
# (5) find the index of the value that eq=median


# to dataframe - median:
roi_climate_median_df = roi_climate_median.to_dataframe() # dimentions: number of catchments squared
roi_climate_median_df = roi_climate_median_df.dropna()
roi_climate_median_df = roi_climate_median_df.drop_duplicates()
# sanity check:
print('length of the df:', len(roi_climate_median_df))

# to dataframe - sum:
# roi_climate_sum_df = roi_climate_sum.to_dataframe() # dimentions: number of catchments squared
# roi_climate_sum_df = roi_climate_sum_df.dropna()
# roi_climate_sum_df = roi_climate_sum_df.drop_duplicates()


 
# roi_climate_median_df.head()

### median value of the not-nan pixels: 

In [ ]:
# since there are even number of pixels calculate medians in space this way: 

def median_ceil(values):
    values = sorted(values)
    values_len = len(values)
    middle = values_len//2
    return values[middle]

# t2m: 
t2m_median = roi_climate_median_df[roi_climate_median_df.t2m==median_ceil(values=roi_climate_median_df.t2m)]
# tp_median = roi_climate_median_df[roi_climate_median_df.tp==median_ceil(values=roi_climate_median_df.tp)]

print('name of the mountain range:', roi.Name, '\n')

print('coordinates for the median temperature:', t2m_median.index, '\n') 


# print('coordinates for the median precipitation:', tp_median.index) # more then one 

In [ ]:
print('name of the mountain range:', roi.Name, '\n')
# t2m_median.t2m.values[0]
t2m_median

In [ ]:
# median temperature coordinates: 

# 0. Eastern Hindu Kush           35.25, 72.75   0.290222
# 1. Weatern Himalaya             32.0,  77.5    -4.169601
# 2. Eastern Himalaya             27.75, 91.0    5.055511
# 3. Central Himalaya             31.25, 78.0    5.66568
# 4. Karakoram                    34.5,  78.5    -13.504562
# 5. Western Pamir                37.25, 72.25   -9.19783
# 6. Pamir Alay                   39.0,  69.0    0.290222
# 7. Northern/Western Tien Shan   43.0,  80.0    3.12915
# 8. Dzhungarsky Alatau           44.75, 79.0    1.454086
# 9. Western Kunlun Shan          36.25, 82.75   -9.67034912109375
# 10. Nyainqentanglha             29.0,  97.0    -2.06459
# 11. Gangdise Mountains          31.75, 81.25   -4.905823
# 12. Hengduan Shan               30.5,  100.75   3.731567
# 13. Tibetan Interior Mountains  32.75, 81.25    -5.162567
# 14. Tanggula Shan               33.75, 91.75    -6.484207
# 15.  Eastern Tibetan Mountains  34.0,  99.75    -4.787506
# 16. Qilian Shan                 37.5,  100.0    -2.851852
# 17. Eastern Kunlun Shan         35.75, 96.5     -6.595551
# 18. Altun Shan                  39.0,  90.25    1.604126
# 19. Eastern Tien Shan           43.75, 92.0      3.453934
# 20. Central Tien Shan           42.75, 83.75     -2.627563
# 21. Eastern Pamir               38.0,  75.5      -6.310211

In [ ]:
# len(median_t2m_regions)
# np.random.rand(21, 1)



In [ ]:
median_t2m_regions = {'Name': ['Eastern Hindu Kush',
                               'Weatern Himalaya','Eastern Himalaya',
                               'Central Himalaya','Karakoram', 'Western Pamir','Pamir Alay', 
                               'Northern/Western Tien Shan','Dzhungarsky Alatau', 'Western Kunlun Shan', 
                               'Nyainqentanglha','Gangdise Mountains','Hengduan Shan','Tibetan Interior Mountains',
                               'Tanggula Shan','Eastern Tibetan Mountains', 'Qilian Shan', 'Eastern Kunlun Shan',
                               'Altun Shan', 'Eastern Tien Shan', 'Central Tien Shan','Eastern Pamir'], 
                      'latitude': [35.25, 32.0, 27.75, 31.25, 34.5, 37.25, 39.0, 43.0, 44.75, 36.25, 29.0, 31.75,
                                   30.5, 32.75, 33.75, 34.0, 37.5, 35.75, 39.0, 43.75, 42.75, 38.0], 
                      'longitude': [72.75, 77.5, 91.0, 78.0, 78.5, 72.25, 69.0, 80.0, 79.0, 82.75, 97.0, 81.25, 
                                    100.75, 81.25, 91.75, 99.75, 100.0, 96.5, 90.25, 92.0, 83.75, 75.5], 
                     't2m_median_2012': [0.290222,-4.169601,5.055511,5.66568,-13.504562,-9.19783,0.290222,3.12915,1.454086,
                                    -9.67034912109375,-2.06459,-4.905823,3.731567,-5.162567,-6.484207,-4.787506,-2.851852,
                                    -6.595551,1.604126,3.453934,-2.627563,-6.310211]}

median_t2m_regions= pd.DataFrame(median_t2m_regions).set_index('Name')

In [ ]:
median_t2m_regions.head() 

### plot points with the outlines pf the regions 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
# hma bounds
hma.boundary.plot(ax = ax, color='black' )
# points
plt.scatter(x = median_t2m_regions.longitude, y = median_t2m_regions.latitude, c = median_t2m_regions.t2m_median_2012)

plt.colorbar()
plt.title('median 2012 temperature')

### get time-series for all these coordinates (temperature) 

## Precipitation

#### annual sum of precipitation -> find median value per region 

In [ ]:
precip = xr.open_dataset("era5_resampled_to_annual/total_precipitation_annual_sum_1979to2020.nc", decode_coords="all")

# ----- crs -----

# add crs to the dataset (netcdf file) - maybe this is not even needed 
precip = precip.rio.write_crs('EPSG:4326')


# ----- median annual sum -------
median = precip.median(dim='year', keep_attrs = True, skipna=True)


In [ ]:

# function for median: 

def median_ceil(values):
    values = sorted(values)
    values_len = len(values)
    middle = values_len//2
    return values[middle]

# median

In [ ]:
''' change name of the mountain range here'''

# hma
roi = hma[hma['Name'] == hma_df.Name[21]]
# name = roi.Name
# print(roi.Name)

In [ ]:
# clip:
clipped = median.rio.clip(roi.geometry, roi.crs, drop=True, invert=False)

# to df:
df = clipped.to_dataframe().reset_index()
df = df.dropna().drop_duplicates()

df = df[df.expver == 1]

# median in space:
median_space = median_ceil(values=df.tp)

row = df[df.tp == median_space]


In [ ]:
print('name of roi:', roi.Name , '\n')

print('lat:', row.latitude.values[0], '\n')
print('lon:', row.longitude.values[0], '\n')
print('tp value:', row.tp.values[0])

In [ ]:
# median precipitation coordinates: 

# 0. Eastern Hindu Kush           36.0  72.25  0.7620535
# 1. Weatern Himalaya             34.25 73.0   1.1706305
# 2. Eastern Himalaya             27.5  89.0   2.7184653
# 3. Central Himalaya             27.5  84.5   1.5944765
# 4. Karakoram                    36.75 73.0   0.58780897
# 5. Western Pamir                36.75 71.25  0.5485622
# 6. Pamir Alay                   40.25 73.75  1.0962509  
# 7. Northern/Western Tien Shan   43.25 78.5   0.935992
# 8. Dzhungarsky Alatau           45.0  80.5   0.7453102
# 9. Western Kunlun Shan          36.0  78.75  0.40251723
# 10. Nyainqentanglha             29.75 91.75  1.0952213
# 11. Gangdise Mountains          31.25 80.75  0.45727164
# 12. Hengduan Shan               30.5  101.25 1.2621787
# 13. Tibetan Interior Mountains  34.5  85.0   0.41629964
# 14. Tanggula Shan               32.75 91.25  0.73406124
# 15.  Eastern Tibetan Mountains  33.25 95.0   0.73050326
# 16. Qilian Shan                 39.0  98.0   0.5301913
# 17. Eastern Kunlun Shan         36.0  96.25  0.40490812
# 18. Altun Shan                  38.75 90.75  0.2197583
# 19. Eastern Tien Shan           43.5  83.0   0.50500554
# 20. Central Tien Shan           41.5  76.75  0.59172446
# 21. Eastern Pamir               37.5  75.25  0.47593403

In [ ]:
annual_tp_regions = {'Name': ['0 Eastern Hindu Kush',
                               '1 Weatern Himalaya','2 Eastern Himalaya',
                               '3 Central Himalaya','4 Karakoram', '5 Western Pamir','6 Pamir Alay', 
                               '7 Northern/Western Tien Shan','8 Dzhungarsky Alatau', '9 Western Kunlun Shan', 
                               '10 Nyainqentanglha','11 Gangdise Mountains','12 Hengduan Shan','13 Tibetan Interior Mountains',
                               '14 Tanggula Shan','15 Eastern Tibetan Mountains', '16 Qilian Shan', '17 Eastern Kunlun Shan',
                               '18 Altun Shan', '19 Eastern Tien Shan', '20 Central Tien Shan','21 Eastern Pamir'], 
                      'latitude': [36.0, 34.25, 27.5, 27.5, 36.75, 36.75, 40.25, 43.25, 45.0, 36.0, 29.75, 
                                   31.25, 30.5, 34.5, 32.75, 33.25, 39.0 , 36.0 , 38.75, 43.5 , 41.5 , 37.5 ], 
                     
                      'longitude': [72.25, 73.0, 89.0, 84.5, 73.0, 71.25, 73.75, 78.5, 80.5, 78.75, 91.75,
                                    80.75, 101.25, 85.0, 91.25, 95.0, 98.0, 96.25, 90.75, 83.0, 76.75, 75.25 ], 
                     
                      'tp_median': [0.7620535, 1.1706305, 2.7184653, 1.5944765, 0.58780897, 0.5485622, 1.0962509, 
                                    0.935992, 0.7453102, 0.40251723, 1.0952213, 0.45727164, 1.2621787 ,0.41629964, 
                                    0.73406124, 0.73050326, 0.5301913, 0.40490812, 0.2197583, 0.50500554, 0.59172446, 
                                    0.47593403]}

median_tp_regions= pd.DataFrame(annual_tp_regions)#.set_index('Name')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
# hma bounds
hma.boundary.plot(ax = ax, color='black' )
# points
plt.scatter(x = median_tp_regions.longitude, y = median_tp_regions.latitude, c = median_tp_regions.tp_median)

plt.colorbar()
plt.title('median annual precipitation sum per region [m]')

plt.savefig('median_annual_precip.png', dpi = 300) 

## time-series for median annual precipitation locations

#### select time_series of climate data for the coordinates of the median points

In [ ]:
# read all climate data for all time: 

t2m_all = xr.open_mfdataset('/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/2m-temperature/*.nc', decode_coords = "all")
tp_all = xr.open_mfdataset('/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/*.nc', decode_coords = "all")
solar_all = xr.open_mfdataset('/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/surface-solar-radiation-downwards/*.nc', decode_coords = "all")
tcc_all = xr.open_mfdataset('/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-cloud-cover/*.nc', decode_coords = "all")

# climate_all = xr.merge([tp_all, t2m_all, solar_all, tcc_all])

#convert temperature K to C
t2m_all['t2m']=t2m_all.t2m-273.15

# precipotation m to mm 
tp_all['tp']=tp_all.tp * 1000

# radiation j/m2 to w/m2
# SSR [W/m2] = SSR [J/m^2] / (3600 seconds)
solar_all['ssrd'] = solar_all.ssrd / 3600



In [ ]:
print('some text,', median_tp_regions.Name[3], ', some other text')

In [ ]:
# change the number of the string here: 
n = 0

In [ ]:
lats_annual =  median_tp_regions.latitude[n]
lons_annual = median_tp_regions.longitude[n]

print(median_tp_regions.Name[n])

In [ ]:
# ---- temperature -------

# select from original climate dataset:
ts = t2m_all.sel(longitude=lons_annual, latitude=lats_annual, method='nearest')
ts = ts.drop('expver')
ts = ts.drop('longitude')
ts = ts.drop('latitude')

## Try to make it run in parallel  

In [ ]:
# ts
from multiprocessing import Pool

pool = Pool()

In [ ]:
pool.map(ts.to_dataframe().dropna().drop_duplicates(), ts)


In [ ]:
pool.close()

In [ ]:
%%time 

ts_df = ts.to_dataframe().dropna().drop_duplicates()
# ts_df = ts.to_dask_dataframe()#.to_csv(

In [ ]:
%%time 

#save .csv file: 
import os.path

save_path = '/Users/varyabazilova/Desktop/hma_regions/out/'
save_file = median_tp_regions.Name[n] + 't2m'

out = os.path.join(save_path, save_file + '.csv') 

ts_df.to_csv(out)


In [ ]:
# calculate how many era5 pix fall into the mountain range polygon

x = ts.t2m.values.flatten()
print('total numnber of pixels within bbox:', x.shape, '\n')

# without nans
x2 = x[~np.isnan(x)]
print('number of not-nan pixels:', x2.shape)

 ### plot time-series for each of this points:

In [ ]:
ts_df = ts.to_dataframe()#.dropna().drop_duplicates()

In [ ]:
ts_df = ts_df.dropna().drop_duplicates()

In [ ]:
ts_df.t2m.plot()
ts_df.tp.plot()


In [ ]:
# ts_df = ts_df.reset_index()

In [ ]:
ts_df.head()
# len(ts_df)

# 193248

In [ ]:
lats_and_lons = ts_df[['latitude', 'longitude']].groupby(['latitude', 'longitude']).size().reset_index()

# print('how many unique pairs of coordinates there are:', len(lats_and_lons))
lats_and_lons

In [ ]:
type(lats_and_lons)

t2m = ts_df[['latitude', 'longitude', 'time', 't2m']]
tp = ts_df[['latitude', 'longitude', 'time', 'tp']]



In [ ]:
#  TEMPERATURES: 

fig, ax = plt.subplots(figsize=(10, 8))

# short = roi.reset_index()

subs_t2m = []

# for lat, lon in zip(roi.latitude, roi.longitude):

for lat, lon in zip(lats_and_lons.latitude, lats_and_lons.longitude):

# for lat, lon in zip(short.latitude.unique(), short.longitude.unique()):
    # print(lat, lon)
    subdf = t2m[(t2m.latitude==lat) & (t2m.longitude==lon)]
    subdf.set_index('time', inplace=True)
    
    subs_t2m.append(subdf)
    
    ax.plot(subdf.index, subdf.t2m, alpha = 0.5, label='{lat} - {lon}'.format(lat=lat, lon=lon))
ax.legend(bbox_to_anchor=(1.04,0.5), loc="center left", ncol=4)
plt.title('t2m')
# plt.suptitle(str(name))

### xr.where() - way
#### spatial dimention (-> value IN SPACE + find location ) 

In [ ]:
# calculate the median value over the region -> vallue IN SPACE 
# then find it's location 
# https://docs.xarray.dev/en/latest/generated/xarray.DataArray.where.html

# roi_t2m_median = roi_climate_median.t2m.median()#dim = axes, keep_attrs = True)


In [ ]:
# with numpy (for T2M and TP): 

# t2m median vallue:
# t2m_median['t2m'].iloc[0] - value from median table: 
median_t2m_where = roi_climate_median.t2m.where(roi_climate_median.t2m == t2m_median['t2m'].iloc[0])


In [ ]:
median_t2m_where.plot()

In [ ]:
# type(roi_climate_median)
# type(median_t2m)
# ds
# t2m_median['t2m'].iloc[0] - t2m_median 

## annual sum of precipitation, then take median vallue of the precipitation sum

In [ ]:
# read file:

annual_tp = xr.open_dataset('/Users/varyabazilova/Desktop/hma_regions/era5_resampled_to_annual/total_precipitation_annual_sum_1979to2020.nc', decode_coords="all")

In [ ]:
# clip with roi of interest
# add crs to the dataset (netcdf file) - maybe this is not even needed 
annual_tp = annual_tp.rio.write_crs('EPSG:4326')

# clip using xarrayrio library:
roi_annual_tp = annual_tp.rio.clip(roi.geometry, roi.crs, drop=True, invert=False)


In [ ]:
# calculate median vallue of in time

annual_median = roi_annual_tp.median(dim = 'year', keep_attrs = True)

In [ ]:
## calculate not nan values 
count = roi_annual_tp.tp[1,:,:].values.flatten()
print('total numnber of pixels within bbox:', count.shape)

# without nans
count2 = count[~np.isnan(count)]
print('number of not-nan pixels:', count2.shape)



In [ ]:
# check for the median: 
if (len(count2) % 2) == 0:
    print ('EVEN number of not-nans -> ceiling/floor median')
else:print ('ODD number of not-nans -> regular median is ok')

In [ ]:
# convert to data frame 
# to dataframe - sum:
annual_median_df = annual_median.to_dataframe() # dimentions: number of catchments squared
annual_median_df = annual_median_df.dropna().drop_duplicates()
# roi_climate_sum_df = roi_climate_sum_df.drop_duplicates()


In [ ]:
# annual_median_df.head()


In [ ]:
# since there are even number of pixels calculate medians in space this way: 

# def median_ceil(values):
    # values = sorted(values)
    # values_len = len(values)
    # middle = values_len//2
    # return values[middle]

# t2m: 
annual_median_xy = annual_median_df[annual_median_df.tp==median_ceil(values=annual_median_df.tp)]

print('region of interest:\n', roi.Name, '\n')

print('coordinates for the median precipitation:\n', annual_median_xy.index) # more then one 



In [ ]:
annual_tp_where = annual_median.tp.where(annual_median.tp == annual_median_xy['tp'].iloc[0])

# annual_tp_where

In [ ]:
annual_tp_where = annual_tp_where.drop('spatial_ref')


In [ ]:
annual_tp_where[1,:,:].plot()



In [ ]:
# resample things somehow? 

# resample for the YEAR resolution: 
# 1. precipitation: (1) take annual sum (2) ???

In [ ]:
# to select mnt-range by name see this: 
# https://github.com/pydata/xarray/issues/501